In [1]:
from GEO_DATA import GeoData
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objs as go
import plotly.express as px
import random

In [ ]:
class MultipleFragment:
    def __init__(self,distance_matrix,cities,cordinates):
        self.distance_matrix = distance_matrix
        self.cities = cities
        self.cordinates = cordinates
        self.n_cities = self.distance_matrix.shape[0]
        self.city_index = {self.cities[i]:i for i in range(self.n_cities)}      
        self.city_index_inv = {i:self.cities[i] for i in range(self.n_cities)}
        
        self.diag_penalty = np.max(self.distance_matrix)*self.n_cities
        self.distance_penalty_matrix = self.distance_matrix.copy()
        for i in range(self.n_cities):
            self.distance_penalty_matrix[i][i] = self.diag_penalty
    
    def num_to_pos(self,n):
        return int(n/self.n_cities),n%self.n_cities
    
    def find_minimum_edge(self):
        idx = np.argmin(self.distance_penalty_matrix)
        a,b = self.num_to_pos(idx)
        self.distance_penalty_matrix[a][b] = self.diag_penalty
        self.distance_penalty_matrix[b][a] = self.diag_penalty
        return a,b
    
    def check_degree_condition(self,G,a,b):
        if G.degree[a] < 2 and G.degree[b] < 2:
            return True
        else:
            return False
    
    def check_cycle(self,G):
        if len(nx.cycle_basis(G.to_undirected())) == 0:
            return False
        else:
            return True
        
    def solve(self):
        G = nx.Graph()

        for i in cordinates:
            lat,lng = cordinates[i]
            G.add_node(i, pos=(lng,lat))
        
        n_edges = 0
        
        while n_edges < self.n_cities:
            a,b = self.find_minimum_edge()
            frm = self.city_index_inv[a]
            to = self.city_index_inv[b]
            #print(a,b)

            if self.check_degree_condition(G,frm,to):
                G.add_edge(frm,to)
                n_edges = len(G.edges)
                if n_edges < n_cities:
                    if self.check_cycle(G):
                        G.remove_edge(frm,to)
                        #print('removed',frm,to)
        tour = nx.cycle_basis(G.to_undirected())[0]
        tour_index = [self.city_index[i] for i in tour]
        first_city = tour_index[0]
        tour_index.append(first_city)
        
        total_dist = self.get_cost(tour_index)
        self.create_graph_plot(tour_index,total_dist)
        
        return tour_index,total_dist
        
        
    def get_cost(self,V):
        s = 0
        for i in range(len(V)-1):
            d = self.distance_matrix[V[i+1]][V[i]]
            s += d
        return s
    
    def create_graph_plot(self,seq,dist):
        G = nx.Graph()
        for i in self.cordinates:
            lat,lng = self.cordinates[i]
            G.add_node(i, pos=(lng,lat))
        

        for i in range(len(seq)-1):
            if i < len(seq)-2:
                frm = self.city_index_inv[seq[i]]
                to = self.city_index_inv[seq[i+1]]
            if i == len(seq) - 2:
                frm = self.city_index_inv[seq[i]]
                to = self.city_index_inv[seq[0]]

            G.add_edge(frm,to)
        
        self.Graph_plotter(G,dist)
            
            
    def Graph_plotter(self,G,dist):
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x0, y0 = G.nodes[edge[0]]['pos']
            x1, y1 = G.nodes[edge[1]]['pos']
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)

        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5, color='#888'),
            hoverinfo='none',
            mode='lines')

        node_x = []
        node_y = []
        for node in G.nodes():
            x, y = G.nodes[node]['pos']
            node_x.append(x)
            node_y.append(y)

        node_trace = go.Scatter(
            x=node_x, y=node_y,
            mode='markers',
            hoverinfo='text',
                line_width=2)
        
        node_adjacencies = []
        
        node_text = []
        for node, adjacencies in enumerate(G.adjacency()):
            node_adjacencies.append(len(adjacencies[1]))
            node_text.append(self.city_index_inv[node])

        node_trace.marker.color = node_adjacencies
        node_trace.text = node_text
        
        fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='TSP Solution - Multiple Fragment'+str(dist),
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
        fig.show()


In [2]:
g = GeoData()
capitals = ['Stuttgart','Munich','Berlin','Potsdam','Bremen','Hamburg','Wiesbaden','Hanover','Schwerin','Düsseldorf','Mainz','Saarbrücken','Dresden','Magdeburg', 'Kiel', 'Erfurt']
distance_matrix = g.get_distance_matrix(capitals)
cordinates = g.get_dict_coordinates(capitals)
city_index = {capitals[i]:i for i in range(len(capitals))}
city_index_inv = {i:capitals[i] for i in range(len(capitals))}
n_cities = len(capitals)

In [8]:
import random

In [11]:
random.randint(0,3)

1

In [19]:
def shuffle(a):
    b = a.copy()
    i = random.randint(0,len(a))
    j = random.randint(0,len(a))
    if j == i:
        j = random.randint(0,len(a))
    
    i_p = b[i]
    j_p = b[j]
    
    b[i] = j_p
    b[j] = i_p
    return b

In [20]:
a = [1,2,3,4,5,6,7,8,9,10]

In [21]:
for _ in range(5):
    b = shuffle(a)
    a = b

In [22]:
a = [1,2,3,4,5,6,7,8,9,10]

In [25]:
a[:3],a[3:7],a[7:]

([1, 2, 3], [4, 5, 6, 7], [8, 9, 10])

In [29]:
def order_crossover(a,b):
    #d = {i : a[i] for i in range(len(a))}
    
    i = random.randint(1,len(a)-1)
    j = random.randint(1,len(a)-1)
    if j == i:
        j = random.randint(0,len(a))
    
    A1 = a[:i]
    B1 = a[i:j]
    C1 = a[j:]
    
    A2 = b[:i]
    B2 = b[i:j]
    C2 = b[j:]
    
    
    A = {}
    B = {}
    
    for k in range(i,j):
        A[k] = b[k]
        B[k] = a[k]
    
    for k in range(i):
        if b[k] not in A:
            A[k] = b[k]
        else:
            A[k] = a[k]
        
        if a[k] not in B:
            B[k] = a[k]
        else:
            B[k] = b[k]
    
    for k in range(j,len(a)):
        if b[k] not in A:
            A[k] = b[k]
        else:
            A[k] = a[k]
        
        if a[k] not in B:
            B[k] = a[k]
        else:
            B[k] = b[k] 
    
    return A,B
        

In [41]:
A,B = order_crossover(a,b)

In [42]:
[A[i] for i in range(len(A))]

[1, 3, 5, 4, 9, 6, 7, 8, 9, 10]

In [40]:
[B[i] for i in range(len(B))]

[1, 2, 5, 4, 9, 6, 7, 8, 9, 10]

In [34]:
A

[1, 2, 5, 4, 9, 6, 7, 8, 9, 10]

In [5]:
d = {i : a[i] for i in range(len(a))}

1

In [ ]:
MF = MultipleFragment(distance_matrix,capitals,cordinates)

In [ ]:
MF.solve()